<a href="https://colab.research.google.com/github/EliasKng/BT-Code/blob/master/Chart_2_text_reduced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chart-2-text-reduced

This notebook will provide the functionality of chart-to-text, however, for single value inputs.

So it will do the data-preparation and then put the values into the model and return the summary for the chart.

The goal is to provide a function:

 **createSummary(chartData: ChartData): string**

where the returned string is the summary

## Startup

### Installations & Imports

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")
% cd /content/gdrive/MyDrive/BA-Code/

! pip install -U spacy
! python3 -m spacy download en_core_web_md



import os
import nltk
import re
from typing import List
import spacy
import io
import sys
import argparse
import numpy as np
from logging import getLogger
import math
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F

nlp = spacy.load('en_core_web_md')
nltk.download('punkt')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/BA-Code
     |████████████████████████████████| 45.7 MB 2.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Utility functions

#### General utility functions

In [4]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self


def word_tokenize(string: str, language: str = "english") -> List[str]:
    """tokenizes a given string into a list of substrings.

    :param string: String to tokenize.
    :param language: Language. Either one of ``english'' or ``german''.
    """
    if language not in ["english", "german"]:
        raise ValueError("language argument has to be either ``english'' or ``german''")

    # excessive whitespaces
    string = re.sub(r"\s+", " ", string)

    # some unicode characters
    string = string.replace("’", "'")
    string = string.replace("”", '"')
    string = string.replace("“", '"')

    # floating point (e.g., 1.3 => 1.3)
    string = re.sub(r"(\d+)\.(\d+)", r"\g<1>._\g<2>", string)

    # percentage (e.g., below.500 => below .500)
    string = re.sub(r"(\w+)\.(\d+)", r"\g<1> ._\g<2>", string)

    # end of quote
    string = string.replace(".``", ". ``")

    # number with apostrophe (e.g. '90)
    string = re.sub(r"\s'(\d+)", r"' \g<1>", string)

    # names with Initials (e.g. C. J. Miles)
    string = re.sub(r"(^|\s)(\w)\. (\w)\.", r"\g<1>\g<2>._ \g<3>._", string)

    # some dots
    string = string.replace("..", " ..")

    # names with apostrophe => expands temporarily
    string = re.sub(r"\w+'(?!d|s|ll|t|re|ve|\s)", r"\g<0>_", string)

    # win-loss scores (German notation seems to be XX:YY, but this is also the time format,
    # and the times are not tokenized in the original RotoWire. So we manually handle XX:YY
    # expression.
    string = re.sub(r"(\d+)-(\d+)", r"\g<1> - \g<2>", string)

    # actual tokenization
    tokenized = nltk.word_tokenize(string, language=language)

    joined = " ".join(tokenized)
    # shrink expanded name-with-apostrophe expressions
    joined = joined.replace("'_", "'")
    # shrink expanded name-with-initial expressions
    joined = joined.replace("._", ".")
    tokenized = joined.split(" ")

    return tokenized

def cleanAxisLabel(label):
    cleanLabel = re.sub('\s', '_', label)
    cleanLabel = cleanLabel.replace('%', '').replace('*', '')
    return cleanLabel
  
def cleanAxisValue(value):
    #print(value)
    if value == '-' or value == 'nan':
        return '0'
    cleanValue = re.sub('\s', '_', value)
    cleanValue = cleanValue.replace('|', '').replace(',', '').replace('%', '').replace('*', '')
    return cleanValue

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

def openMultiColumnData(df):
    cols = df.columns
    size = df.shape[0]
    return cols, size
  
def getSubject(titleTokens, nerEntities):
    fillers = ['in', 'the', 'and', 'or', 'an', 'as', 'can', 'be', 'a', ':', '-',
              'to', 'but', 'is', 'of', 'it', 'on', '.', 'at', '(', ')', ',', ';']
    entities = {}
    entities['Subject'] = []
    entities['Date'] = []
    # manually find dates, it performs better than using NER
    for word in titleTokens:
        if word.isnumeric():
            if len(word) > 3:
                entities['Date'].append(word)
        elif word.replace('/', '').isnumeric():
            word = word.split('/')[0]
            if len(word) > 3:
                entities['Date'].append(word)
        elif word.replace('-', '').isnumeric():
            word = word.split('-')[0]
            if len(word) > 3:
                entities['Date'].append(word)
    # get named entites from title
    for X in nerEntities:
        if X.label_ == 'GPE' or X.label_ == 'ORG' or X.label_ == 'NORP' or X.label_ == 'LOC':
            cleanSubject = [word for word in X.text.split() if word.isalpha() and word not in fillers]
            if len(cleanSubject) > 0:
                entities['Subject'].append(' '.join(cleanSubject))
        if len(entities['Date']) < 1:
            if X.label_ == 'DATE':
                if X.text.isnumeric():
                    entities['Date'].append(X.text)
    # guess subject if NER doesn't find one
    if len(entities['Subject']) == 0:
        uppercaseWords = [word for word in titleTokens if word[0].isupper()]
        if len(uppercaseWords) > 1:
            guessedSubject = ' '.join(uppercaseWords[1:])
        else:
            guessedSubject = uppercaseWords[0]
        entities['Subject'].append(guessedSubject)
    # print(entities['Date'])
    cleanTitle = [titleWord for titleWord in titleTokens if titleWord.lower() not in fillers]
    return entities, cleanTitle


#### Dictionary

In [5]:
logger = getLogger()


BOS_WORD = '<s>'
EOS_WORD = '</s>'
PAD_WORD = '<pad>'
UNK_WORD = '<unk>'

SPECIAL_WORD = '<special%i>'
SPECIAL_WORDS = 10

SEP_WORD = SPECIAL_WORD % 0
MASK_WORD = SPECIAL_WORD % 1


class Dictionary(object):

    def __init__(self, id2word, word2id):
        assert len(id2word) == len(word2id)
        self.id2word = id2word
        self.word2id = word2id
        self.bos_index = word2id[BOS_WORD]
        self.eos_index = word2id[EOS_WORD]
        self.pad_index = word2id[PAD_WORD]
        self.unk_index = word2id[UNK_WORD]
        self.check_valid()

    def __len__(self):
        """
        Returns the number of words in the dictionary.
        """
        return len(self.id2word)

    def __getitem__(self, i):
        """
        Returns the word of the specified index.
        """
        return self.id2word[i]

    def __contains__(self, w):
        """
        Returns whether a word is in the dictionary.
        """
        return w in self.word2id

    def __eq__(self, y):
        """
        Compare this dictionary with another one.
        """
        self.check_valid()
        y.check_valid()
        if len(self.id2word) != len(y):
            return False
        return all(self.id2word[i] == y[i] for i in range(len(y)))

    def check_valid(self):
        """
        Check that the dictionary is valid.
        """
        assert self.bos_index == 0
        assert self.eos_index == 1
        assert self.pad_index == 2
        assert self.unk_index == 3
        assert all(self.id2word[4 + i] == SPECIAL_WORD % i for i in range(SPECIAL_WORDS))
        assert len(self.id2word) == len(self.word2id)
        for i in range(len(self.id2word)):
            assert self.word2id[self.id2word[i]] == i

    def index(self, word, no_unk=False):
        """
        Returns the index of the specified word.
        """
        if no_unk:
            return self.word2id[word]
        else:
            return self.word2id.get(word, self.unk_index)

    def max_vocab(self, max_vocab):
        """
        Limit the vocabulary size.
        """
        assert max_vocab >= 1
        init_size = len(self)
        self.id2word = {k: v for k, v in self.id2word.items() if k < max_vocab}
        self.word2id = {v: k for k, v in self.id2word.items()}
        self.check_valid()
        logger.info("Maximum vocabulary size: %i. Dictionary size: %i -> %i (removed %i words)."
                    % (max_vocab, init_size, len(self), init_size - len(self)))

    @staticmethod
    def read_vocab(vocab_path):
        """
        Create a dictionary from a vocabulary file.
        """
        skipped = 0
        assert os.path.isfile(vocab_path), vocab_path
        word2id = {BOS_WORD: 0, EOS_WORD: 1, PAD_WORD: 2, UNK_WORD: 3}
        for i in range(SPECIAL_WORDS):
            word2id[SPECIAL_WORD % i] = 4 + i
        f = open(vocab_path, 'r', encoding='utf-8')
        for i, line in enumerate(f):
            if '\u2028' in line:
                skipped += 1
                continue
            fields = line.rstrip().split()
            if len(fields) <= 0 or len(fields) > 2:
                skipped += 1
                continue

            if fields[0] in word2id:
                skipped += 1
                print('%s already in vocab' % fields[0])
                continue
            word2id[fields[0]] = 4 + SPECIAL_WORDS + i - skipped  # shift because of extra words
        f.close()
        id2word = {v: k for k, v in word2id.items()}
        dico = Dictionary(id2word, word2id)
        logger.info("Read %i words from the vocabulary file." % len(dico))
        if skipped > 0:
            logger.warning("Skipped %i empty lines!" % skipped)
        return dico

    @staticmethod
    def index_data(path, bin_path, dico):
        """
        Index sentences with a dictionary.
        """
        if bin_path is not None and os.path.isfile(bin_path):
            print("Loading dataOld from %s ..." % bin_path)
            data = torch.load(bin_path)
            assert dico == data['dico']
            return data

        positions = []
        sentences = []
        unk_words = {}

        # index sentences
        f = open(path, 'r', encoding='utf-8')
        for i, line in enumerate(f):
            if i % 1000000 == 0 and i > 0:
                print(i)
            s = line.rstrip().split()
            # skip empty sentences
            if len(s) == 0:
                print("Empty sentence in line %i." % i)
            # index sentence words
            count_unk = 0
            indexed = []
            for w in s:
                word_id = dico.index(w, no_unk=False)
                # if we find a special word which is not an unknown word, skip the sentence
                if 0 <= word_id < 4 + SPECIAL_WORDS and word_id != 3:
                    logger.warning('Found unexpected special word "%s" (%i)!!' % (w, word_id))
                    continue
                assert word_id >= 0
                indexed.append(word_id)
                if word_id == dico.unk_index:
                    unk_words[w] = unk_words.get(w, 0) + 1
                    count_unk += 1
            # add sentence
            positions.append([len(sentences), len(sentences) + len(indexed)])
            sentences.extend(indexed)
            sentences.append(1)  # EOS index
        f.close()

        # tensorize dataOld
        positions = np.int64(positions)
        if len(dico) < 1 << 16:
            sentences = np.uint16(sentences)
        elif len(dico) < 1 << 31:
            sentences = np.int32(sentences)
        else:
            raise Exception("Dictionary is too big.")

        assert sentences.min() >= 0
        data = {
            'dico': dico,
            'positions': positions,
            'sentences': sentences,
            'unk_words': unk_words,
        }
        if bin_path is not None:
            print("Saving the dataOld to %s ..." % bin_path)
            torch.save(data, bin_path, pickle_protocol=4)

        return data

    @staticmethod
    def index_table(table_path, table_label_path, table_dico, bin_path):
        """
        """
        if bin_path is not None and os.path.isfile(bin_path):
            print("Loading dataOld from %s ..." % bin_path)
            data = torch.load(bin_path)
            assert table_dico == data['dico']
            print("%s checked. Nothing is done." % bin_path)
            return data

        positions = []
        table_seq_entity = []
        table_seq_type = []
        table_seq_value = []
        table_seq_feat = []
        table_seq_label = []

        # index table_contents
        table_inf = open(table_path, 'r', encoding='utf-8')
        table_label_inf = open(table_label_path, 'r', encoding='utf-8')

        for i, (line, label_line) in enumerate(zip(table_inf, table_label_inf)):
            table_items = line.strip().split()
            table_label = label_line.strip().split()
            assert len(table_items) == len(table_label)
            # skip empty table_contents
            if len(table_items) == 0:
                print("Empty sentence in line %i." % i)
                continue

            # entity, type, value, feat
            table_entity_indexed = []
            table_type_indexed = []
            table_value_indexed = []
            table_feat_indexed = []

            table_label_indexed = []

            for item, label in zip(table_items, table_label):
                fields = item.split('|')
                assert len(fields) == 4
                entity_id = table_dico.index(fields[0], no_unk=False)
                type_id = table_dico.index(fields[1], no_unk=False)
                value_id = table_dico.index(fields[2], no_unk=False)
                feat_id = table_dico.index(fields[3], no_unk=False)

                label_id = int(label)

                table_entity_indexed.append(entity_id)
                table_type_indexed.append(type_id)
                table_value_indexed.append(value_id)
                table_feat_indexed.append(feat_id)

                table_label_indexed.append(label_id)

            # add sentence
            positions.append([len(table_seq_entity), len(table_seq_entity) + len(table_entity_indexed)])
            table_seq_entity.extend(table_entity_indexed)
            table_seq_entity.append(1)  # EOS index

            table_seq_type.extend(table_type_indexed)
            table_seq_type.append(0)  # empty feat

            table_seq_value.extend(table_value_indexed)
            table_seq_value.append(0)

            table_seq_feat.extend(table_feat_indexed)
            table_seq_feat.append(0)

            table_seq_label.extend(table_label_indexed)
            table_seq_label.append(0)

        table_inf.close()
        table_label_inf.close()

        # tensorize dataOld
        positions = np.int64(positions)
        if len(table_dico) < 1 << 16:
            table_seq_entity = np.uint16(table_seq_entity)
            table_seq_type = np.uint16(table_seq_type)
            table_seq_value = np.uint16(table_seq_value)
            table_seq_feat = np.uint16(table_seq_feat)
        elif len(table_dico) < 1 << 31:
            table_seq_entity = np.int32(table_seq_entity)
            table_seq_type = np.uint32(table_seq_type)
            table_seq_value = np.uint32(table_seq_value)
            table_seq_feat = np.uint32(table_seq_feat)
        else:
            raise Exception("Dictionary is too big.")

        table_seq_label = np.uint8(table_seq_label)

        data = {
            'dico': table_dico,
            'positions': positions,
            'table_entities': table_seq_entity,
            'table_types': table_seq_type,
            'table_values': table_seq_value,
            'table_feats': table_seq_feat,
            'table_labels': table_seq_label,
        }
        if bin_path is not None:
            print("Saving the dataOld to %s ..." % bin_path)
            torch.save(data, bin_path, pickle_protocol=4)

        return data

    @staticmethod
    def index_summary(summary_path, summary_label_path, dico, bin_path, max_len=600):
        """
        Index summaries with a dictionary.
        """
        if bin_path is not None and os.path.isfile(bin_path):
            print("Loading dataOld from %s ..." % bin_path)
            data = torch.load(bin_path)
            assert dico == data['dico']
            print("%s checked. Nothing is done." % bin_path)
            return data

        positions = []
        summaries = []
        summary_labels = []

        # index summaries
        summary_inf = open(summary_path, 'r', encoding='utf-8')
        summary_label_inf = open(summary_label_path, 'r', encoding='utf-8')

        for i, (summary_line, label_line) in enumerate(zip(summary_inf, summary_label_inf)):
            if i % 1000000 == 0 and i > 0:
                print(i)
            summary_tokens = summary_line.rstrip().split()
            summary_token_labels = label_line.rstrip().split()
            assert len(summary_tokens) == len(summary_token_labels)
            # skip empty summaries
            if len(summary_tokens) == 0:
                print("Empty sentence in line %i." % i)
                continue
            # index sentence words
            summary_indexed = []
            summary_label_indexed = []
            for token, label in zip(summary_tokens, summary_token_labels):
                word_id = dico.index(token, no_unk=False)
                label_id = int(label)
                # if we find a special word which is not an unknown word, skip the sentence
                if 0 <= word_id < 4 + SPECIAL_WORDS and word_id != 3:
                    logger.warning('Found unexpected special word "%s" (%i)!!' % (token, word_id))
                    continue
                assert word_id >= 0

                summary_indexed.append(word_id)
                summary_label_indexed.append(label_id)
            if len(summary_indexed) > max_len:
                summary_indexed = summary_indexed[:max_len]
                summary_label_indexed = summary_label_indexed[:max_len]

            # add sentence
            positions.append([len(summaries), len(summaries) + len(summary_indexed)])
            summaries.extend(summary_indexed)
            summaries.append(1)  # EOS index
            summary_labels.extend(summary_label_indexed)
            summary_labels.append(0)

        summary_inf.close()
        summary_label_inf.close()

        # tensorize dataOld
        positions = np.int64(positions)
        if len(dico) < 1 << 16:
            summaries = np.uint16(summaries)
        elif len(dico) < 1 << 31:
            summaries = np.int32(summaries)
        else:
            raise Exception("Dictionary is too big.")

        summary_labels = np.uint8(summary_labels)

        assert summaries.min() >= 0
        data = {
            'dico': dico,
            'positions': positions,
            'summaries': summaries,
            'summary_labels': summary_labels,
        }
        if bin_path is not None:
            print("Saving the dataOld to %s ..." % bin_path)
            torch.save(data, bin_path, pickle_protocol=4)

        return data

#### Transformer Deconder & Transformer encoder

In [6]:
N_MAX_POSITIONS = 602  # maximum summary sequence length

DECODER_ONLY_PARAMS = [
    'layer_norm15.%i.weight', 'layer_norm15.%i.bias',
    'encoder_attn.%i.q_lin.weight', 'encoder_attn.%i.q_lin.bias',
    'encoder_attn.%i.k_lin.weight', 'encoder_attn.%i.k_lin.bias',
    'encoder_attn.%i.v_lin.weight', 'encoder_attn.%i.v_lin.bias',
    'encoder_attn.%i.out_lin.weight', 'encoder_attn.%i.out_lin.bias'
]

TRANSFORMER_LAYER_PARAMS = [
    'attentions.%i.q_lin.weight', 'attentions.%i.q_lin.bias',
    'attentions.%i.k_lin.weight', 'attentions.%i.k_lin.bias',
    'attentions.%i.v_lin.weight', 'attentions.%i.v_lin.bias',
    'attentions.%i.out_lin.weight', 'attentions.%i.out_lin.bias',
    'layer_norm1.%i.weight', 'layer_norm1.%i.bias',
    'ffns.%i.lin1.weight', 'ffns.%i.lin1.bias',
    'ffns.%i.lin2.weight', 'ffns.%i.lin2.bias',
    'layer_norm2.%i.weight', 'layer_norm2.%i.bias'
]


logger = getLogger()

def smoothed_softmax_cross_entropy_with_logits(logits, labels, smoothing=0.0):
    if not smoothing:
        loss = F.cross_entropy(logits, labels, reduction='mean')
        return loss

    vocab_size = logits.size(1)
    n = (vocab_size - 1)
    p = 1.0 - smoothing
    q = smoothing / n

    one_hot = torch.randn(1, vocab_size, device=logits.device)
    one_hot.fill_(q)
    soft_targets = one_hot.repeat(labels.view(-1, 1).size(0), 1)
    soft_targets.scatter_(1, labels.view(-1, 1), p)

    log_prb = F.log_softmax(logits, dim=1)
    loss = -(soft_targets * log_prb).sum(dim=1)
    loss = loss.sum() / len(labels)
    return loss

def Embedding(num_embeddings, embedding_dim, padding_idx=None):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.normal_(m.weight, mean=0, std=embedding_dim ** -0.5)
    if padding_idx is not None:
        nn.init.constant_(m.weight[padding_idx], 0)
    return m


def Linear(in_features, out_features, bias=True):
    m = nn.Linear(in_features, out_features, bias)
    # nn.init.normal_(m.weight, mean=0, std=1)
    # nn.init.xavier_uniform_(m.weight)
    # nn.init.constant_(m.bias, 0.)
    return m


def create_sinusoidal_embeddings(n_pos, dim, out):
    position_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / dim) for j in range(dim)]
        for pos in range(n_pos)
    ])
    out[:, 0::2] = torch.FloatTensor(np.sin(position_enc[:, 0::2]))
    out[:, 1::2] = torch.FloatTensor(np.cos(position_enc[:, 1::2]))
    out.detach_()
    out.requires_grad = False


def gelu(x):
    """
    GELU activation
    https://arxiv.org/abs/1606.08415
    https://github.com/huggingface/pytorch-openai-transformer-lm/blob/master/model_pytorch.py#L14
    https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/modeling.py
    """
    # return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    return 0.5 * x * (1.0 + torch.erf(x / math.sqrt(2.0)))


def get_masks(slen, lengths, causal):
    """
    Generate hidden states mask, and optionally an attention mask.
    """
    assert lengths.max().item() <= slen
    bs = lengths.size(0)
    alen = torch.arange(slen, dtype=torch.long, device=lengths.device)
    mask = alen < lengths[:, None]

    # attention mask is the same as mask, or triangular inferior attention (causal)
    if causal:
        attn_mask = alen[None, None, :].repeat(bs, slen, 1) <= alen[None, :, None]
    else:
        attn_mask = mask

    # sanity check
    assert mask.size() == (bs, slen)
    assert causal is False or attn_mask.size() == (bs, slen, slen)

    return mask, attn_mask

class BinaryOutputLayer(nn.Module):
    """
    0/1 Classification layer (binary classification).
    """
    def __init__(self, params):
        super().__init__()
        self.pad_index = params.pad_index
        dim = params.emb_dim
        self.proj = Linear(dim, 1, bias=True)
        self.proj_act = nn.Sigmoid()
        self.criterion = nn.BCELoss()

    def forward(self, x, y):
        """
        Compute the loss, and optionally the scores.
        """
        scores = self.proj(x)
        scores = self.proj_act(scores)
        y = y.view_as(scores).float()
        loss = self.criterion(scores, y)
        return scores, loss

    def get_scores(self, x):
        """
        Compute scores.
        """
        scores = self.proj(x)
        scores = self.proj_act(scores)
        return scores


class PredLayer(nn.Module):
    """
    Prediction layer (cross_entropy or adaptive_softmax).
    """
    def __init__(self, params):
        super().__init__()
        self.tgt_n_words = params.tgt_n_words
        self.pad_index = params.pad_index
        dim = params.emb_dim
        try:
            self.label_smoothing = params.label_smoothing
        except:
            self.label_smoothing = 0.0

        self.proj = Linear(dim, self.tgt_n_words, bias=True)

    def forward(self, x, y):
        """
        Compute the loss, and optionally the scores.
        """
        assert (y == self.pad_index).sum().item() == 0
        logits = self.proj(x).view(-1, self.tgt_n_words)
        loss = smoothed_softmax_cross_entropy_with_logits(logits, y, self.label_smoothing)

        return logits, loss

    def get_scores(self, x):
        """
        Compute scores.
        """
        return self.proj(x)


class MultiHeadAttention(nn.Module):

    NEW_ID = itertools.count()

    def __init__(self, n_heads, dim, dropout):
        super().__init__()
        self.layer_id = next(MultiHeadAttention.NEW_ID)
        self.dim = dim
        self.n_heads = n_heads
        self.dropout = dropout
        assert self.dim % self.n_heads == 0

        self.q_lin = Linear(dim, dim)
        self.k_lin = Linear(dim, dim)
        self.v_lin = Linear(dim, dim)
        self.out_lin = Linear(dim, dim)

    def forward(self, input, mask, kv=None, cache=None):
        """
        Self-attention (if kv is None) or attention over source sentence (provided by kv).
        """
        # Input is (bs, qlen, dim)
        # Mask is (bs, klen) (non-causal) or (bs, klen, klen)
        bs, qlen, dim = input.size()
        if kv is None:
            klen = qlen if cache is None else cache['slen'] + qlen
        else:
            klen = kv.size(1)
        assert dim == self.dim, 'Dimensions do not match: %s input vs %s configured' % (dim, self.dim)
        n_heads = self.n_heads
        dim_per_head = dim // n_heads
        mask_reshape = (bs, 1, qlen, klen) if mask.dim() == 3 else (bs, 1, 1, klen)

        def shape(x):
            """  projection """
            return x.view(bs, -1, self.n_heads, dim_per_head).transpose(1, 2)

        def unshape(x):
            """  compute context """
            return x.transpose(1, 2).contiguous().view(bs, -1, self.n_heads * dim_per_head)

        q = shape(self.q_lin(input))                                          # (bs, n_heads, qlen, dim_per_head)
        if kv is None:
            k = shape(self.k_lin(input))                                      # (bs, n_heads, qlen, dim_per_head)
            v = shape(self.v_lin(input))                                      # (bs, n_heads, qlen, dim_per_head)
        elif cache is None or self.layer_id not in cache:
            k = v = kv
            k = shape(self.k_lin(k))                                          # (bs, n_heads, qlen, dim_per_head)
            v = shape(self.v_lin(v))                                          # (bs, n_heads, qlen, dim_per_head)

        if cache is not None:
            if self.layer_id in cache:
                if kv is None:
                    k_, v_ = cache[self.layer_id]
                    k = torch.cat([k_, k], dim=2)                             # (bs, n_heads, klen, dim_per_head)
                    v = torch.cat([v_, v], dim=2)                             # (bs, n_heads, klen, dim_per_head)
                else:
                    k, v = cache[self.layer_id]
            cache[self.layer_id] = (k, v)

        q = q / math.sqrt(dim_per_head)                                       # (bs, n_heads, qlen, dim_per_head)
        scores = torch.matmul(q, k.transpose(2, 3))                           # (bs, n_heads, qlen, klen)
        mask = (mask == 0).view(mask_reshape).expand_as(scores)               # (bs, n_heads, qlen, klen)
        scores.masked_fill_(mask, -float('inf'))                              # (bs, n_heads, qlen, klen)

        weights = F.softmax(scores.float(), dim=-1).type_as(scores)           # (bs, n_heads, qlen, klen)
        weights = F.dropout(weights, p=self.dropout, training=self.training)  # (bs, n_heads, qlen, klen)
        context = torch.matmul(weights, v)                                    # (bs, n_heads, qlen, dim_per_head)
        context = unshape(context)                                            # (bs, qlen, dim)

        return self.out_lin(context)


class TransformerFFN(nn.Module):

    def __init__(self, in_dim, dim_hidden, out_dim, dropout, gelu_activation):
        super().__init__()
        self.dropout = dropout
        self.lin1 = Linear(in_dim, dim_hidden)
        self.lin2 = Linear(dim_hidden, out_dim)
        self.act = gelu if gelu_activation else F.relu

    def forward(self, input):
        x = self.lin1(input)
        x = self.act(x)
        x = self.lin2(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        return x


class TransformerEncoder(nn.Module):

    def __init__(self, params, dico, with_output):
        """
        Transformer model (encoder or decoder).
        """
        super().__init__()

        self.with_output = with_output
        self.with_positional_emb = params.encoder_positional_emb

        self.n_words = params.src_n_words

        self.eos_index = params.eos_index
        self.pad_index = params.pad_index
        self.dico = dico
        assert len(self.dico) == self.n_words

        # model parameters
        self.emb_dim = params.emb_dim // 4  # 128 by default
        self.dim = params.emb_dim       # 512 by default
        self.hidden_dim = self.dim * 4  # 2048 by default
        self.n_heads = params.n_heads   # 8 by default
        self.n_layers = params.enc_n_layers
        self.dropout = params.dropout
        self.attention_dropout = params.attention_dropout
        assert self.dim % self.n_heads == 0, 'transformer dim must be a multiple of n_heads'

        # embeddings
        # TODO remove the hardcode number
        if params.encoder_positional_emb:
            self.position_embeddings = Embedding(800, self.dim)
            if params.sinusoidal_embeddings:
                create_sinusoidal_embeddings(800, self.dim, out=self.position_embeddings.weight)
        self.embeddings = Embedding(self.n_words, self.emb_dim, padding_idx=self.pad_index)
        self.layer_norm_emb = nn.LayerNorm(self.dim, eps=1e-12)

        # transformer layers
        self.attentions = nn.ModuleList()
        self.layer_norm1 = nn.ModuleList()
        self.ffns = nn.ModuleList()
        self.layer_norm2 = nn.ModuleList()

        for _ in range(self.n_layers):
            self.attentions.append(MultiHeadAttention(self.n_heads, self.dim, dropout=self.attention_dropout))
            self.layer_norm1.append(nn.LayerNorm(self.dim, eps=1e-12))
            self.ffns.append(TransformerFFN(self.dim, self.hidden_dim, self.dim, dropout=self.dropout, gelu_activation=params.gelu_activation))
            self.layer_norm2.append(nn.LayerNorm(self.dim, eps=1e-12))

        # output layer
        self.pred_layer = BinaryOutputLayer(params)

    def forward(self, mode, **kwargs):
        """
        Forward function with different forward modes.
        ### Small hack to handle PyTorch distributed.
        """
        if mode == 'fwd':
            return self.fwd(**kwargs)
        elif mode == 'predict':
            return self.predict(**kwargs)
        elif mode == 'score':
            return self.get_scores(**kwargs)
        else:
            raise Exception("Unknown mode: %s" % mode)

    def fwd(self, x1, x2, x3, x4, lengths):
        """
        Inputs:
            `x` LongTensor(slen, bs), containing word indices
            `lengths` LongTensor(bs), containing the length of each sentence
            `causal` Boolean, if True, the attention is only done over previous hidden states
        """
        # lengths = (x != self.pad_index).float().sum(dim=1)
        # mask = x != self.pad_index

        # check inputs
        slen, bs = x1.size()
        assert lengths.size(0) == bs
        assert lengths.max().item() <= slen
        x1 = self.embeddings(x1.transpose(0, 1))
        x2 = self.embeddings(x2.transpose(0, 1))
        x3 = self.embeddings(x3.transpose(0, 1))
        x4 = self.embeddings(x4.transpose(0, 1))

        # generate masks
        mask, attn_mask = get_masks(slen, lengths, causal=False)

        positions = x1.new(slen).long()
        positions = torch.arange(slen, out=positions).unsqueeze(0)

        # embeddings
        tensor = torch.cat((x1, x2, x3, x4), dim=-1)
        if self.with_positional_emb:
            tensor = tensor + self.position_embeddings(positions).expand_as(tensor)
        #tensor = x1 + x2 + x3 + x4
        tensor = self.layer_norm_emb(tensor)
        tensor = F.dropout(tensor, p=self.dropout, training=self.training)
        tensor *= mask.unsqueeze(-1).to(tensor.dtype)

        # transformer layers
        for i in range(self.n_layers):

            # self attention
            attn = self.attentions[i](tensor, attn_mask)
            attn = F.dropout(attn, p=self.dropout, training=self.training)
            tensor = tensor + attn
            tensor = self.layer_norm1[i](tensor)

            # FFN
            tensor = tensor + self.ffns[i](tensor)
            tensor = self.layer_norm2[i](tensor)
            tensor *= mask.unsqueeze(-1).to(tensor.dtype)

        # move back sequence length to dimension 0
        tensor = tensor.transpose(0, 1)

        return tensor

    def predict(self, tensor, pred_mask, y):
        """
        Given the last hidden state, compute word scores and/or the loss.
            `pred_mask` is a ByteTensor of shape (slen, bs), filled with 1 when
                we need to predict a word
            `y` is a LongTensor of shape (pred_mask.sum(),)
        """
        masked_tensor = tensor[pred_mask.unsqueeze(-1).expand_as(tensor)].view(-1, self.dim)
        scores, loss = self.pred_layer(masked_tensor, y)
        return scores, loss

    def get_scores(self, tensor):
        scores = self.pred_layer.get_scores(tensor)
        return scores


class TransformerDecoder(nn.Module):

    def __init__(self, params, dico, with_output):
        """
        Transformer model (encoder or decoder).
        """
        super().__init__()

        # encoder / decoder, output layer
        self.with_output = with_output

        # dictionary / languages
        self.n_words = params.tgt_n_words
            
        self.eos_index = params.eos_index
        self.pad_index = params.pad_index
        self.dico = dico
        print(self.n_words)
        print(len(self.dico))
        assert len(self.dico) == (self.n_words)

        # model parameters
        self.dim = params.emb_dim       # 512 by default
        self.hidden_dim = self.dim * 4  # 2048 by default
        self.n_heads = params.n_heads   # 8 by default
        self.n_layers = params.dec_n_layers
        self.dropout = params.dropout
        self.attention_dropout = params.attention_dropout
        assert self.dim % self.n_heads == 0, 'transformer dim must be a multiple of n_heads'

        # embeddings
        self.position_embeddings = Embedding(N_MAX_POSITIONS, self.dim)
        if params.sinusoidal_embeddings:
            create_sinusoidal_embeddings(N_MAX_POSITIONS, self.dim, out=self.position_embeddings.weight)
        self.embeddings = Embedding(self.n_words, self.dim, padding_idx=self.pad_index)
        self.layer_norm_emb = nn.LayerNorm(self.dim, eps=1e-12)

        # transformer layers
        self.attentions = nn.ModuleList()
        self.layer_norm1 = nn.ModuleList()
        self.ffns = nn.ModuleList()
        self.layer_norm2 = nn.ModuleList()
        self.layer_norm15 = nn.ModuleList()
        self.encoder_attn = nn.ModuleList()

        for _ in range(self.n_layers):
            self.attentions.append(MultiHeadAttention(self.n_heads, self.dim, dropout=self.attention_dropout))
            self.layer_norm1.append(nn.LayerNorm(self.dim, eps=1e-12))
            self.layer_norm15.append(nn.LayerNorm(self.dim, eps=1e-12))
            self.encoder_attn.append(MultiHeadAttention(self.n_heads, self.dim, dropout=self.attention_dropout))
            self.ffns.append(TransformerFFN(self.dim, self.hidden_dim, self.dim, dropout=self.dropout, gelu_activation=params.gelu_activation))
            self.layer_norm2.append(nn.LayerNorm(self.dim, eps=1e-12))

        if self.with_output:
            self.pred_layer = PredLayer(params)
            if params.share_inout_emb:
                self.pred_layer.proj.weight = self.embeddings.weight

    def forward(self, mode, **kwargs):
        """
        Forward function with different forward modes.
        ### Small hack to handle PyTorch distributed.
        """
        if mode == 'fwd':
            return self.fwd(**kwargs)
        elif mode == 'predict':
            return self.predict(**kwargs)
        else:
            raise Exception("Unknown mode: %s" % mode)

    def fwd(self, x, lengths, causal, src_enc=None, src_len=None, positions=None, cache=None):
        """
        Inputs:
            `x` LongTensor(slen, bs), containing word indices
            `lengths` LongTensor(bs), containing the length of each sentence
            `causal` Boolean, if True, the attention is only done over previous hidden states
            `positions` LongTensor(slen, bs), containing word positions
        """
        # lengths = (x != self.pad_index).float().sum(dim=1)
        # mask = x != self.pad_index

        # check inputs
        slen, bs = x.size()
        assert lengths.size(0) == bs
        assert lengths.max().item() <= slen
        x = x.transpose(0, 1)  # batch size as dimension 0
        assert (src_enc is None) == (src_len is None)
        if src_enc is not None:
            assert src_enc.size(0) == bs, "{}!={}".format(src_enc.size(0), bs)

        # generate masks
        mask, attn_mask = get_masks(slen, lengths, causal)
        if src_enc is not None:
            src_mask = torch.arange(src_enc.size(1), dtype=torch.long, device=lengths.device) < src_len[:, None]

        # positions
        if positions is None:
            positions = x.new(slen).long()
            positions = torch.arange(slen, out=positions).unsqueeze(0)
        else:
            assert positions.size() == (slen, bs), positions.size()
            positions = positions.transpose(0, 1)

        # do not recompute cached elements
        if cache is not None:
            _slen = slen - cache['slen']
            x = x[:, -_slen:]
            positions = positions[:, -_slen:]
            mask = mask[:, -_slen:]
            attn_mask = attn_mask[:, -_slen:]

        # embeddings
        tensor = self.embeddings(x)
        tensor = tensor + self.position_embeddings(positions).expand_as(tensor)
        tensor = self.layer_norm_emb(tensor)
        tensor = F.dropout(tensor, p=self.dropout, training=self.training)
        tensor *= mask.unsqueeze(-1).to(tensor.dtype)

        # transformer layers
        for i in range(self.n_layers):

            # self attention
            attn = self.attentions[i](tensor, attn_mask, cache=cache)
            attn = F.dropout(attn, p=self.dropout, training=self.training)
            tensor = tensor + attn
            tensor = self.layer_norm1[i](tensor)

            # encoder attention (for decoder only)
            if src_enc is not None:
                attn = self.encoder_attn[i](tensor, src_mask, kv=src_enc, cache=cache)
                attn = F.dropout(attn, p=self.dropout, training=self.training)
                tensor = tensor + attn
                tensor = self.layer_norm15[i](tensor)

            # FFN
            tensor = tensor + self.ffns[i](tensor)
            tensor = self.layer_norm2[i](tensor)
            tensor *= mask.unsqueeze(-1).to(tensor.dtype)

        # update cache length
        if cache is not None:
            cache['slen'] += tensor.size(1)

        # move back sequence length to dimension 0
        tensor = tensor.transpose(0, 1)

        return tensor

    def predict(self, tensor, pred_mask, y):
        """
        Given the last hidden state, compute word scores and/or the loss.
            `pred_mask` is a ByteTensor of shape (slen, bs), filled with 1 when
                we need to predict a word
            `y` is a LongTensor of shape (pred_mask.sum(),)
        """
        masked_tensor = tensor[pred_mask.unsqueeze(-1).expand_as(tensor)].view(-1, self.dim)
        scores, loss = self.pred_layer(masked_tensor, y)
        return scores, loss

    def generate(self, src_enc, src_len, max_len=200, sample_temperature=None, vocab_mask=None):
        """
        Decode a sentence given initial start.
        `x`:
            - LongTensor(bs, slen)
                <EOS> W1 W2 W3 <EOS> <PAD>
                <EOS> W1 W2 W3   W4  <EOS>
        `lengths`:
            - LongTensor(bs) [5, 6]
        `positions`:
            - False, for regular "arange" positions (LM)
            - True, to reset positions from the new generation (MT)
        """

        # input batch
        bs = len(src_len)
        assert src_enc.size(0) == bs

        # generated sentences
        generated = src_len.new(max_len, bs)  # upcoming output
        generated.fill_(self.pad_index)       # fill upcoming ouput with <PAD>
        generated[0].fill_(self.eos_index)    # we use <EOS> for <BOS> everywhere

        # positions
        positions = src_len.new(max_len).long()
        positions = torch.arange(max_len, out=positions).unsqueeze(1).expand(max_len, bs)

        # current position / max lengths / length of generated sentences / unfinished sentences
        cur_len = 1
        gen_len = src_len.clone().fill_(1)
        unfinished_sents = src_len.clone().fill_(1)

        # cache compute states
        cache = {'slen': 0}

        while cur_len < max_len:

            # compute word scores
            tensor = self.forward(
                'fwd',
                x=generated[:cur_len],
                lengths=gen_len,
                positions=positions[:cur_len],
                causal=True,
                src_enc=src_enc,
                src_len=src_len,
                cache=cache
            )
            assert tensor.size() == (1, bs, self.dim)
            tensor = tensor.data[-1, :, :]               # (bs, dim)
            scores = self.pred_layer.get_scores(tensor)  # (bs, n_words)


            # select next words: sample or greedy
            if sample_temperature is None:
                next_words = torch.topk(scores, 1)[1].squeeze(1)
            else:
                next_words = torch.multinomial(F.softmax(scores / sample_temperature, dim=1), 1).squeeze(1)
            assert next_words.size() == (bs,)

            # update generations / lengths / finished sentences / current length
            generated[cur_len] = next_words * unfinished_sents + self.pad_index * (1 - unfinished_sents)
            gen_len.add_(unfinished_sents)
            unfinished_sents.mul_(next_words.ne(self.eos_index).long())
            cur_len = cur_len + 1

            # stop when there is a </s> in each sentence, or if we exceed the maximul length
            if unfinished_sents.max() == 0:
                break

        # add <EOS> to unfinished sentences
        if cur_len == max_len:
            generated[-1].masked_fill_(unfinished_sents.byte(), self.eos_index)

        # sanity check
        assert (generated == self.eos_index).sum() == 2 * bs

        return generated[:cur_len], gen_len

    def generate_beam(self, src_enc, src_len, beam_size, length_penalty, early_stopping, max_len=200):
        """
        Decode a sentence given initial start.
        `x`:
            - LongTensor(bs, slen)
                <EOS> W1 W2 W3 <EOS> <PAD>
                <EOS> W1 W2 W3   W4  <EOS>
        `lengths`:
            - LongTensor(bs) [5, 6]
        `positions`:
            - False, for regular "arange" positions (LM)
            - True, to reset positions from the new generation (MT)
        """

        # check inputs
        assert src_enc.size(0) == src_len.size(0)
        assert beam_size >= 1

        # batch size / number of words
        bs = len(src_len)
        n_words = self.n_words

        # expand to beam size the source latent representations / source lengths
        src_enc = src_enc.unsqueeze(1).expand((bs, beam_size) + src_enc.shape[1:]).contiguous().view((bs * beam_size,) + src_enc.shape[1:])
        src_len = src_len.unsqueeze(1).expand(bs, beam_size).contiguous().view(-1)

        # generated sentences (batch with beam current hypotheses)
        generated = src_len.new(max_len, bs * beam_size)  # upcoming output
        generated.fill_(self.pad_index)                   # fill upcoming ouput with <PAD>
        generated[0].fill_(self.eos_index)                # we use <EOS> for <BOS> everywhere

        # generated hypotheses
        generated_hyps = [BeamHypotheses(beam_size, max_len, length_penalty, early_stopping) for _ in range(bs)]

        # positions
        positions = src_len.new(max_len).long()
        positions = torch.arange(max_len, out=positions).unsqueeze(1).expand_as(generated)

        # scores for each sentence in the beam
        beam_scores = src_enc.new(bs, beam_size).fill_(0)
        beam_scores[:, 1:] = -1e9
        beam_scores = beam_scores.view(-1)

        # current position
        cur_len = 1

        # cache compute states
        cache = {'slen': 0}

        # done sentences
        done = [False for _ in range(bs)]

        while cur_len < max_len:

            # compute word scores
            tensor = self.forward(
                'fwd',
                x=generated[:cur_len],
                lengths=src_len.new(bs * beam_size).fill_(cur_len),
                positions=positions[:cur_len],
                causal=True,
                src_enc=src_enc,
                src_len=src_len,
                cache=cache
            )
            assert tensor.size() == (1, bs * beam_size, self.dim)
            tensor = tensor.data[-1, :, :]               # (bs * beam_size, dim)
            #print(tensor)
            scores = self.pred_layer.get_scores(tensor)  # (bs * beam_size, n_words)
            scores = F.log_softmax(scores, dim=-1)       # (bs * beam_size, n_words)
            assert scores.size() == (bs * beam_size, n_words)

            # select next words with scores
            #scores of n top sentences where n = beam_size
            _scores = scores + beam_scores[:, None].expand_as(scores)  # (bs * beam_size, n_words)
            #print(f'1 {_scores}')
            #top score?
            _scores = _scores.view(bs, beam_size * n_words)            # (bs, beam_size * n_words)
            #print(f'2 {_scores}')
            next_scores, next_words = torch.topk(_scores, 2 * beam_size, dim=1, largest=True, sorted=True)
            #test = []
            #for idx, value in zip(next_words[0], next_scores[0]):
                # get beam and word IDs
                #beam_id = idx // n_words
                #word_id = idx % n_words
                #test.append(self.dico[word_id.item()])
                #print(next_score, next_word)
                #if next_word in removedDict:
                    #print(next_word)
            #print(f'next words: {test}')
            assert next_scores.size() == next_words.size() == (bs, 2 * beam_size)

            # next batch beam content
            # list of (bs * beam_size) tuple(next hypothesis score, next word, current position in the batch)
            next_batch_beam = []

            # for each sentence
            for sent_id in range(bs):

                # if we are done with this sentence
                done[sent_id] = done[sent_id] or generated_hyps[sent_id].is_done(next_scores[sent_id].max().item())
                if done[sent_id]:
                    next_batch_beam.extend([(0, self.pad_index, 0)] * beam_size)  # pad the batch
                    continue

                # next sentence beam content
                next_sent_beam = []

                # next words for this sentence
                for idx, value in zip(next_words[sent_id], next_scores[sent_id]):

                    # get beam and word IDs
                    beam_id = idx // n_words
                    word_id = idx % n_words
                    #print(, value)
                    #print('loop')

                    # end of sentence, or next word
                    if word_id == self.eos_index or cur_len + 1 == max_len:
                        generated_hyps[sent_id].add(generated[:cur_len, sent_id * beam_size + beam_id].clone(), value.item())
                        #print(f'hyp: {generated_hyps}')
                    else:
                        next_sent_beam.append((value, word_id, sent_id * beam_size + beam_id))
                        #if word_id.item() in self.dico.id2word.keys():
                            #if 'template' in self.dico.id2word[word_id.item()]:
                                #if word_id.item() not in [ids[1] for ids in next_sent_beam]:
                                    #print(self.dico.id2word[word_id.item()])
                                    #next_sent_beam.append((value, word_id, sent_id * beam_size + beam_id))
                                #else:
                                #    print(f'{self.dico.id2word[word_id.item()]} : already in : {[self.dico.id2word[ids[1].item()] for ids in next_sent_beam]}')
                            #else:

                        #else:
                            #print(f'bad {word_id.item()}')

                    # the beam for next step is full
                    if len(next_sent_beam) == beam_size:
                        break
                """
                for i in range(len(next_sent_beam)):
                    #print(self.dico[word_id.item()])
                    if self.dico[word_id.item()] in removedDict:
                        print(next_sent_beam)
                        #if predicted template is not valid, pop the tuple and append it to the end
                        badBeam = next_sent_beam[i].pop()
                        next_sent_beam.append(badBeam)
                        print(next_sent_beam)
                        """
                # update next beam content
                assert len(next_sent_beam) == 0 if cur_len + 1 == max_len else beam_size
                if len(next_sent_beam) == 0:
                    next_sent_beam = [(0, self.pad_index, 0)] * beam_size  # pad the batch
                next_batch_beam.extend(next_sent_beam)
                assert len(next_batch_beam) == beam_size * (sent_id + 1)

            # sanity check / prepare next batch
            assert len(next_batch_beam) == bs * beam_size
            beam_scores = beam_scores.new([x[0] for x in next_batch_beam])
            beam_words = generated.new([x[1] for x in next_batch_beam])
            beam_idx = src_len.new([x[2] for x in next_batch_beam])

            #decoded = [self.dico[x.item()] for x in beam_words]
            #print(f'decoded: {decoded}')

            # re-order batch and internal states
            generated = generated[:, beam_idx]
            generated[cur_len] = beam_words
            #print(generated)
            #print(generated[cur_len])
            for k in cache.keys():
                if k != 'slen':
                    cache[k] = (cache[k][0][beam_idx], cache[k][1][beam_idx])

            # update current length
            cur_len = cur_len + 1

            # stop when we are done with each sentence
            if all(done):
                break

        # visualize hypotheses
        # print([len(x) for x in generated_hyps], cur_len)
        # globals().update( locals() );
        # !import code; code.interact(local=vars())
        #for ii in range(bs):
        #    for ss, ww in sorted(generated_hyps[ii].hyp, key=lambda x: x[0], reverse=True):
        #        print("%.3f " % ss + " ".join(self.dico[x] for x in ww.tolist()))
        #        print("")

        # select the best hypotheses
        tgt_len = src_len.new(bs)
        best = []

        for i, hypotheses in enumerate(generated_hyps):
            best_hyp = max(hypotheses.hyp, key=lambda x: x[0])[1]
            tgt_len[i] = len(best_hyp) + 1  # +1 for the <EOS> symbol
            best.append(best_hyp)

        # generate target batch
        decoded = src_len.new(tgt_len.max().item(), bs).fill_(self.pad_index)
        for i, hypo in enumerate(best):
            decoded[:tgt_len[i] - 1, i] = hypo
            decoded[tgt_len[i] - 1, i] = self.eos_index

        # sanity check
        assert (decoded == self.eos_index).sum() == 2 * bs

        return decoded, tgt_len

    def generate_slot(self, x, xlen, y_type, src_enc, src_len, sample_temperature=None):
        # input batch
        bs = len(src_len)
        assert src_enc.size(0) == bs, "{}!={}".format(src_enc.size(0), bs)
        max_len = xlen.max()

        indices = (x==4).nonzero()
        pred_indices, batch_indices = torch.split(indices, 1, 1)

        # generated sentences
        generated = x.clone()

        # positions
        positions = torch.arange(max_len, dtype=torch.long, device=src_len.device)
        positions = positions.unsqueeze(1).expand(max_len, bs)

        # current position / max lengths / length of generated sentences / unfinished sentences
        cur_idx = 0
        max_idx = pred_indices.size(0)

        while cur_idx < max_idx:
            cur_len = pred_indices[cur_idx].item()
            batch_inx = batch_indices[cur_idx].item()

            cur_type = y_type[cur_len,batch_inx]
            # compute word scores
            tensor = self.forward(
                'fwd',
                x=generated[:cur_len, batch_inx][:,None],
                lengths=pred_indices[cur_idx],
                positions=positions[:cur_len, batch_inx][:,None],
                causal=True,
                src_enc=src_enc[batch_inx][None],
                src_len=src_len[batch_inx][None],
                cache=None
            )
            #assert tensor.size() == (1, bs, self.dim)
            tensor = tensor.data[-1, :, :]               # (bs, dim)
            tensor_type_embedding = self.prediction_type_embeddings(cur_type)
            tensor = torch.cat((tensor, tensor_type_embedding.unsqueeze(0)), dim=-1)
            tensor = self.prediction_type_pooler(tensor)
            scores = self.pred_layer.get_scores(tensor)  # (bs, n_words)


            # select next words: sample or greedy
            if sample_temperature is None:
                # take top-2 to avoid generating <eos>
                next_words = torch.topk(scores, 2)[1].squeeze()
            else:
                next_words = torch.multinomial(F.softmax(scores / sample_temperature, dim=1), 1).squeeze(1)
            assert next_words.size() == (2,), next_words.size()

            # update generations / lengths / finished sentences / current length
            # generated[cur_len, batch_inx] = next_words
            generated[cur_len, batch_inx] = next_words[0] if next_words[0] != self.eos_index else next_words[1]
            cur_idx = cur_idx + 1

        # sanity check
        assert (generated == self.eos_index).sum() == 2 * bs, (generated == self.eos_index).sum()

        return generated

    def generate_slot_beam(self, x, xlen, y_type, src_enc, src_len, beam_size=2):
        # input batch
        bs = len(src_len)
        n_words = self.n_words
        assert src_enc.size(0) == bs, "{}!={}".format(src_enc.size(0), bs)
        max_len = xlen.max()

        # expand to beam size the source latent representations / source lengths
        src_enc = src_enc.unsqueeze(1).expand((bs, beam_size) + src_enc.shape[1:]).contiguous()
        src_len = src_len.unsqueeze(1).expand(bs, beam_size).contiguous()
        
        result = []
        for batch_idx in range(bs):
            x_i = x[:,batch_idx]
            pred_indices = (x_i==4).nonzero()

            src_enc_sub = src_enc[batch_idx]
            src_len_sub = src_len[batch_idx]

            # generated sentences
            generated = x_i.clone()
            generated = generated.unsqueeze(1).expand(max_len, beam_size)

            positions = torch.arange(max_len, dtype=torch.long, device=src_len.device)
            positions = positions.unsqueeze(1).expand(max_len, beam_size)

            # scores for each sentence in the beam
            beam_scores = src_enc.new(1, beam_size).fill_(0)
            beam_scores[:, 1:] = -1e9
            beam_scores = beam_scores.view(-1)

            cur_len = 1
            cache = {'slen': 0}

            while cur_len < max_len:
                cur_type = y_type[cur_len, batch_idx]
                # compute word scores
                tensor = self.forward(
                    'fwd',
                    x=generated[:cur_len],
                    lengths=src_len.new(beam_size).fill_(cur_len),
                    positions=positions[:cur_len],
                    causal=True,
                    src_enc=src_enc_sub,
                    src_len=src_len_sub,
                    cache=cache
                )
                assert tensor.size() == (1, beam_size, self.dim)
                tensor = tensor.data[-1, :, :]               # (beam_size, dim)
                if cur_type.item() == 0:
                    tgt_id = x_i[cur_len].item()
                    scores = self.pred_layer.get_scores(tensor)  # (beam_size, n_words)
                    scores = scores[:, tgt_id] # get the target token score
                    scores = F.log_softmax(scores, dim=-1)       # (beam_size)
                    assert scores.size() == (beam_size,)
                    
                    # here we don't need to sort the beam order
                    beam_scores = beam_scores + scores
                else:
                    tensor_type = self.prediction_type_embeddings(cur_type)
                    tensor_type = tensor_type.unsqueeze(0).expand_as(tensor)
                    tensor = torch.cat((tensor, tensor_type), dim=-1)
                    tensor = self.prediction_type_pooler(tensor)
                    scores = self.pred_layer.get_scores(tensor)  # (beam_size, n_words)
                    scores = F.log_softmax(scores, dim=-1)       # (beam_size, n_words)
                    assert scores.size() == (beam_size, n_words)

                    # select next words with scores
                    _scores = scores + beam_scores[:, None].expand_as(scores)  # (beam_size, n_words)
                    _scores = _scores.view(1, beam_size * n_words)             # (1, beam_size * n_words)

                    next_scores, next_words = torch.topk(_scores, 2 * beam_size, dim=1, largest=True, sorted=True)
                    assert next_scores.size() == next_words.size() == (1, 2 * beam_size)

                    # next sentence beam content
                    next_sent_beam = []

                    for idx, value in zip(next_words[0], next_scores[0]):
                        beam_id = idx // n_words
                        word_id = idx % n_words

                        # end of sentence, or next word
                        if word_id != self.eos_index:
                            next_sent_beam.append((value, word_id, beam_id))

                        # the beam for next step is full
                        if len(next_sent_beam) == beam_size:
                            break

                    beam_scores = beam_scores.new([x[0] for x in next_sent_beam])
                    beam_words  = generated.new([x[1] for x in next_sent_beam])
                    beam_idx = src_len.new([x[2] for x in next_sent_beam])

                    generated = generated[:, beam_idx]
                    generated[cur_len] = beam_words
            
                    for k in cache.keys():
                        if k != 'slen':
                            cache[k] = (cache[k][0][beam_idx], cache[k][1][beam_idx])
                cur_len += 1
            # sanity check
            assert (generated == self.eos_index).sum() == 2 * beam_size, (generated == self.eos_index).sum()
            max_score, index = beam_scores.max(0)
            result.append(generated[:,index].unsqueeze(1))

        result = torch.cat(result, dim=1)
        assert result.size() == x.size()

        return result


class BeamHypotheses(object):

    def __init__(self, n_hyp, max_len, length_penalty, early_stopping):
        """
        Initialize n-best list of hypotheses.
        """
        self.max_len = max_len - 1  # ignoring <BOS>
        self.length_penalty = length_penalty
        self.early_stopping = early_stopping
        self.n_hyp = n_hyp
        self.hyp = []
        self.worst_score = 1e9

    def __len__(self):
        """
        Number of hypotheses in the list.
        """
        return len(self.hyp)

    def add(self, hyp, sum_logprobs):
        """
        Add a new hypothesis to the list.
        """
        #print(self.removedDict)
        score = sum_logprobs / len(hyp) ** self.length_penalty
        if len(self) < self.n_hyp or score > self.worst_score:
            #if hyp not in self.removedDict:
                #print(score, hyp)
            self.hyp.append((score, hyp))
            if len(self) > self.n_hyp:
                sorted_scores = sorted([(s, idx) for idx, (s, _) in enumerate(self.hyp)])
                del self.hyp[sorted_scores[0][1]]
                self.worst_score = sorted_scores[1][0]
            else:
                self.worst_score = min(score, self.worst_score)

    def is_done(self, best_sum_logprobs):
        """
        If there are enough hypotheses and that none of the hypotheses being generated
        can become better than the worst one in the heap, then we are done with this sentence.
        """
        if len(self) < self.n_hyp:
            return False
        elif self.early_stopping:
            return True
        else:
            return self.worst_score >= best_sum_logprobs / self.max_len ** self.length_penalty

### Titles

In [7]:
def clean_title(title):
  cleanedTitle = word_tokenize(title)
  # replace (2009 - 2016) with (2009 to 2016)
  lastTokens = cleanedTitle[-3:]
  if lastTokens[1] == '-' and lastTokens[0].isnumeric() and lastTokens[2].isnumeric():
    cleanedTitle[-2] = 'to'
  cleanedTitle = ' '.join(cleanedTitle).replace('*', '')
  return cleanedTitle

###Preprocessing
- Converts data tables into a sequence of records (taken as input by the model): `data/*split*/trainData.txt`
- Cleans summary tokens and substitutes any possible tokens with data variables(e.g., 2018 -> templateValue[0][0]): `data/*split*/trainSummary.txt`
- Cleans the title tokens: `data/*split*/trainTitle.txt`
- Labels the occurrences of records mentioned within the summary: `data/*split*/trainDataLabel.txt`
- Labels the summary tokens which match a record: `data/*split*/trainSummaryLabel.txt`
- Saves the gold summaries: `data/*split*/testOriginalSummary.txt`

In [8]:
def preprocessData(df, chartType = 'bar_chart'):
  complexChartTypes = []

  cols, size = openMultiColumnData(df)
  complexChartTypes.append(chartType)
  cleanCols = [cleanAxisLabel(axis) for axis in cols]
  dataLine = ''
  colData = []
  for col in df:
      vals = df[col].values
      cleanVals = [cleanAxisValue(str(value)) for value in vals]
      colData.append(cleanVals)
  # iterate through each table row
  for m in range(0, size):
      axisTypes = []
      #rowData = []
      dataLabels = []
      for axis, n in zip(cols, range(cols.size)):
          if is_number(axis[0]):
              axisTypes.append('numerical')
          else:
              axisTypes.append('categorical')
          value = str(df.at[m, axis])
          cleanValue = cleanAxisValue(value)
          #rowData.append(cleanValue)
          record = f"{cleanCols[n]}|{cleanValue}|{n}|{chartType}"
          dataLine += f'{record} '
          dataLabels.append([0 for item in range(size)])

  return dataLine

## Generate summary

In [9]:
def generateSummary(titleLine, tableDataLine):
    params = argparse.Namespace(batch_size=8, 
                              beam_size=4, 
                              early_stopping=False, 
                              length_penalty=1.0, 
                              model_path='./models/aug17-80.pth',
                              output_path='output.txt')
    assert os.path.isfile(params.model_path)

    with torch.no_grad():
        # generate parser / parse parameters
        #parser = get_parser()
        #params = parser.parse_args()
        reloaded = torch.load(params.model_path)

        model_params = AttrDict(reloaded['params'])
        

        # update dictionary parameters
        for name in ['src_n_words', 'tgt_n_words', 'bos_index', 'eos_index', 'pad_index', 'unk_index', 'mask_index']:
            setattr(params, name, getattr(model_params, name))
        # print(f'src {getattr(model_params, "src_n_words")}')
        # print(f'tgt {getattr(model_params, "tgt_n_words")}')
        # build dictionary / build encoder / build decoder / reload weights
        source_dico = Dictionary(reloaded['source_dico_id2word'], reloaded['source_dico_word2id'])
        target_dico = Dictionary(reloaded['target_dico_id2word'], reloaded['target_dico_word2id'])
        # originalDecoder = reloaded['decoder'].copy()
        encoder = TransformerEncoder(model_params, source_dico, with_output=False)
        encoder.load_state_dict(reloaded['encoder'])
        decoder = TransformerDecoder(model_params, target_dico, with_output=True)
        decoder.load_state_dict(reloaded['decoder'])
        # read sentences from stdin
        table_lines = [tableDataLine]
        title_lines = [titleLine]



        #Do we have as many Titles as Diagram data?
        assert len(title_lines) == len(table_lines)

        #Opens the output file
        outf = io.open(params.output_path, 'w', encoding='utf-8')

        fillers = ['in', 'the', 'and', 'or', 'an', 'as', 'can', 'be', 'a', ':', '-',
                  'to', 'but', 'is', 'of', 'it', 'on', '.', 'at', '(', ')', ',', 'with']

        #Do for all lines incrementing with batch_size
        for i in range(0, len(table_lines), params.batch_size):
            # prepare batch

            """valueLengths = []
            xLabelLengths = []
            yLabelLengths = []
            titleLengths = []"""
            enc_x1_ids = []
            enc_x2_ids = []
            enc_x3_ids = []
            enc_x4_ids = []
            for table_line, title_line in zip(table_lines[i:i + params.batch_size], title_lines[i:i + params.batch_size]):
                record_seq = [each.split('|') for each in table_line.split()]
                assert all([len(x) == 4 for x in record_seq])

                enc_x1_ids.append(torch.LongTensor([source_dico.index(x[0]) for x in record_seq]))
                enc_x2_ids.append(torch.LongTensor([source_dico.index(x[1]) for x in record_seq]))
                enc_x3_ids.append(torch.LongTensor([source_dico.index(x[2]) for x in record_seq]))
                enc_x4_ids.append(torch.LongTensor([source_dico.index(x[3]) for x in record_seq]))

                xLabel = record_seq[1][0].split('_')
                yLabel = record_seq[0][0].split('_')
                """cleanXLabel = len([item for item in xLabel if item not in fillers])
                cleanYLabel = len([item for item in yLabel if item not in fillers])
                cleanTitle = len([word for word in title_line.split() if word not in fillers])

                xLabelLengths.append(cleanXLabel)
                yLabelLengths.append(cleanYLabel)
                titleLengths.append(cleanTitle)
                valueLengths.append(round(len(record_seq)/2))"""

            enc_xlen = torch.LongTensor([len(x) + 2 for x in enc_x1_ids])
            enc_x1 = torch.LongTensor(enc_xlen.max().item(), enc_xlen.size(0)).fill_(params.pad_index)
            enc_x1[0] = params.eos_index
            enc_x2 = torch.LongTensor(enc_xlen.max().item(), enc_xlen.size(0)).fill_(params.pad_index)
            enc_x2[0] = params.eos_index
            enc_x3 = torch.LongTensor(enc_xlen.max().item(), enc_xlen.size(0)).fill_(params.pad_index)
            enc_x3[0] = params.eos_index
            enc_x4 = torch.LongTensor(enc_xlen.max().item(), enc_xlen.size(0)).fill_(params.pad_index)
            enc_x4[0] = params.eos_index

            for j, (s1,s2,s3,s4) in enumerate(zip(enc_x1_ids, enc_x2_ids, enc_x3_ids, enc_x4_ids)):
                if enc_xlen[j] > 2:  # if sentence not empty
                    enc_x1[1:enc_xlen[j] - 1, j].copy_(s1)
                    enc_x2[1:enc_xlen[j] - 1, j].copy_(s2)
                    enc_x3[1:enc_xlen[j] - 1, j].copy_(s3)
                    enc_x4[1:enc_xlen[j] - 1, j].copy_(s4)
                enc_x1[enc_xlen[j] - 1, j] = params.eos_index
                enc_x2[enc_xlen[j] - 1, j] = params.eos_index
                enc_x3[enc_xlen[j] - 1, j] = params.eos_index
                enc_x4[enc_xlen[j] - 1, j] = params.eos_index

            enc_x1 = enc_x1.cuda()
            enc_x2 = enc_x2.cuda()
            enc_x3 = enc_x3.cuda()
            enc_x4 = enc_x4.cuda()
            enc_xlen = enc_xlen.cuda()

            # encode source batch and translate it
            encoder_output = encoder('fwd', x1=enc_x1, x2=enc_x2, x3=enc_x3, x4=enc_x4, lengths=enc_xlen)
            encoder_output = encoder_output.transpose(0, 1)

            max_len = 602
            if params.beam_size <= 1:
                decoded, dec_lengths = decoder.generate(encoder_output, enc_xlen, max_len=max_len)
            elif params.beam_size > 1:
                decoded, dec_lengths = decoder.generate_beam(encoder_output, enc_xlen, params.beam_size, 
                                                params.length_penalty, params.early_stopping, max_len=max_len)

            for j in range(decoded.size(1)):
                # remove delimiters
                sent = decoded[:, j]
                delimiters = (sent == params.eos_index).nonzero().view(-1)
                assert len(delimiters) >= 1 and delimiters[0].item() == 0
                sent = sent[1:] if len(delimiters) == 1 else sent[1:delimiters[1]]
                # print(sent)
                # output translation
                # source = table_lines[i + j].strip()
                # print(source)
                tokens = []
                for k in range(len(sent)):
                    ids = sent[k].item()
                    #if ids in removedDict:
                    #    print('index error')
                    word = target_dico[ids]
                    tokens.append(word)
                target = " ".join(tokens)
                sys.stderr.write("%i / %i: %s\n" % (i + j, len(table_lines), target))
                outf.write(target + "\n")
        outf.close()

##Testing

In [10]:
testTitles = ['Facebook: number of monthly active users worldwide 2008-2019','National Basketball Association all-time scoring leaders 1946-2020','Instagram accounts with the most followers worldwide 2020']


# Import pandas library
import pandas as pd
from io import StringIO

TESTDATA = StringIO("""Country,Average number of children per woman
Singapore,1.38
Republic of Korea,1.44
United Arab Emirates,1.45
Puerto Rico,1.45
Bosnia and Herzegovina,1.47
Saint Lucia,1.48
Greece,1.5
Cyprus,1.51
Italy,1.51
Republic of Moldova,1.52
"China, Taiwan Province of China",1.53
Albania,1.53
Mauritius,1.54
Thailand,1.54
Qatar,1.56
Nepal,1.56
Croatia,1.56
Japan,1.57
Serbia,1.57
Brazil,1.58
North Macedonia,1.58
Brunei Darussalam,1.59
Portugal,1.59
Spain,1.59
Canada,1.59
    """)

df = pd.read_csv(TESTDATA, sep=",")
  
# print dataframe.
cleanedTitle = clean_title('Countries with the lowest fertility rate globally 2050-2055')
dataline = preprocessData(df)

generateSummary(cleanedTitle, dataline)

11198
11198


RuntimeError: ignored

In [12]:
% cd gdrive/MyDrive/BA-Code/

/content/gdrive/MyDrive/BA-Code


In [13]:
ls

Chart_2_text_reduced.ipynb  models/  output.txt


In [ ]:
cd BA-Code

[Errno 2] No such file or directory: 'BA-Code'
/content/gdrive


In [1]:
import torch
print(torch.__version__)

1.5.0


In [19]:
! pip install torch==1.5.0 torchvision==0.6.0

     |████████████████████████████████| 752.0 MB 10 kB/s 
     |████████████████████████████████| 6.6 MB 1.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.5.0 which is incompatible.
